# 1. Install the model and libraries

In [1]:
#install onnx runtime
!pip install --pre onnxruntime-genai --q

StatementMeta(, 0583015f-4ecc-438e-a4f8-096ccd962bdd, 3, Finished, Available, Finished)

In [2]:
#create a directory for the model
import os 
import shutil
import onnxruntime_genai as og

model_path = '/lakehouse/default/Files/phi3mini'

#Mount a lakehouse first
if not os.path.exists(model_path):
    os.mkdir(model_path)
    print(f"model will be downloaded to {model_path}")
else:
    print(f"{model_path} exists")

StatementMeta(, 0583015f-4ecc-438e-a4f8-096ccd962bdd, 4, Finished, Available, Finished)

model will be downloaded to /lakehouse/default/Files/phi3mini


In [3]:
!huggingface-cli download microsoft/Phi-3-mini-4k-instruct-onnx --include cpu_and_mobile/cpu-int4-rtn-block-32-acc-level-4/* --local-dir ./lakehouse/default/Files/phi3mini

StatementMeta(, 0583015f-4ecc-438e-a4f8-096ccd962bdd, 5, Finished, Available, Finished)

Fetching 10 files:   0%|                                 | 0/10 [00:00<?, ?it/s]Downloading 'cpu_and_mobile/cpu-int4-rtn-block-32-acc-level-4/tokenizer.json' to 'lakehouse/default/Files/phi3mini/.huggingface/download/cpu_and_mobile/cpu-int4-rtn-block-32-acc-level-4/tokenizer.json.efc309ef56b8d8fba1b50d1b4a6e5be6cfded459.incomplete'

(…)-rtn-block-32-acc-level-4/tokenizer.json:   0%|  | 0.00/1.84M [00:00<?, ?B/s]Downloading 'cpu_and_mobile/cpu-int4-rtn-block-32-acc-level-4/phi3-mini-4k-instruct-cpu-int4-rtn-block-32-acc-level-4.onnx.data' to 'lakehouse/default/Files/phi3mini/.huggingface/download/cpu_and_mobile/cpu-int4-rtn-block-32-acc-level-4/phi3-mini-4k-instruct-cpu-int4-rtn-block-32-acc-level-4.onnx.data.5db30ce699aee1123cf9045742488db5928006fa618a42cb3c0840322a85ad0f.incomplete'


(…)k-32-acc-level-4/special_tokens_map.json: 100%|█| 568/568 [00:00<00:00, 3.04M
Download complete. Moving file to lakehouse/default/Files/phi3mini/cpu_and_mobile/cpu-int4-rtn-block-32-acc-level-4/specia

In [4]:
#copy files to a lakehouse from the temp directory
source_dir = os.path.abspath(".") + model_path
destination_dir = model_path

try:
    shutil.copytree(source_dir, destination_dir, dirs_exist_ok=True)
    print("Directory copied successfully.")
except Exception as e:
    print(f"Error occurred: {e}")

StatementMeta(, 0583015f-4ecc-438e-a4f8-096ccd962bdd, 6, Finished, Available, Finished)

Directory copied successfully.


In [5]:
#confirm files have been copied
for f in mssparkutils.fs.ls('Files/phi3mini/cpu_and_mobile/cpu-int4-rtn-block-32-acc-level-4'):
    print(f.name)

StatementMeta(, 0583015f-4ecc-438e-a4f8-096ccd962bdd, 7, Finished, Available, Finished)

added_tokens.json
config.json
configuration_phi3.py
genai_config.json
phi3-mini-4k-instruct-cpu-int4-rtn-block-32-acc-level-4.onnx
phi3-mini-4k-instruct-cpu-int4-rtn-block-32-acc-level-4.onnx.data
special_tokens_map.json
tokenizer.json
tokenizer.model
tokenizer_config.json


# 2. Generate email responses for 5 most negative ratings

In [7]:
df = spark.sql("SELECT * FROM Augmented.customer_feedback.hotel_reviews \
WHERE sentiment = 'negative' AND reviews_rating = 1 ORDER BY confidence_negative DESC LIMIT 5")

# Function to process reviews and return categories
def generate_response(review_text):
    # Initialize model and tokenizer for each call
    model = og.Model(model_path + '/cpu_and_mobile/cpu-int4-rtn-block-32-acc-level-4')
    tokenizer = og.Tokenizer(model)

    instruction = "You are an expert reviewer. Propose an email response to the customer who shared the review. \
    Politely express your gratitude and point out how the company intends to improve the poor customer experience mentioned in the review. \
    Be conise."
    prompt = f"<|system|>{instruction}<|end|><|user|>{review_text}<|end|><|assistant|>"

    # Tokenize prompt and generate response
    input_tokens = tokenizer.encode(prompt)
    
    params = og.GeneratorParams(model)
    params.try_graph_capture_with_max_batch_size(1)
    params.set_search_options(max_length=1024, temperature=0.6)
    
    params.input_ids = input_tokens
    
    generator = og.Generator(model, params)
    
    # Collect all generated tokens
    generated_tokens = []
    
    while not generator.is_done():
        generator.compute_logits()
        generator.generate_next_token()
        new_token = generator.get_next_tokens()[0]
        generated_tokens.append(new_token)

    # Decode all generated tokens into a final string
    final_output = tokenizer.decode(generated_tokens).strip()
    
    return final_output

# Collect results into a list
results = []
for row in df.collect():
    review_text = row['reviews_text']
    response = generate_response(review_text)  # Call the categorization function
    results.append((review_text, response))

# Create a new DataFrame with results
results_df = spark.createDataFrame(results, ["reviews_text", "response"])

# Save results
results_df.write.option("overwrite", "true").saveAsTable("Augmented.customer_feedback.hotel_reviews_responses")

StatementMeta(, 0583015f-4ecc-438e-a4f8-096ccd962bdd, 9, Finished, Available, Finished)

In [8]:
%%sql
SELECT * FROM Augmented.customer_feedback.hotel_reviews_responses;

StatementMeta(, 0583015f-4ecc-438e-a4f8-096ccd962bdd, 10, Finished, Available, Finished)

<Spark SQL result set with 5 rows and 2 fields>